# Strategy 5: limiting interval sizes

### Formulation of the problem and notation

This strategy re-samples points and applies adaptive using the average at each point. Every time we ask for a new point, the learner can decide to sample a new point or re-sample an existing one to improve the estimate of the mean value.

The function we will sample is given by 
<br>
<br>
<center>$g(x) = f(x)\cdot\xi_\text{mul}(x) + \xi_\text{add}(x)$,</center>
<br>
where $f(x)$ is the true value of the underlying function and $\xi_\text{mul}(x)$ $\xi_\text{add}(x)$ are random variables (multiplicative and additive noise, respectively). Let us assume that both types of noise follow a Gaussian distribution: $\xi_\text{mul}(x)\sim\mathcal{N}(1,\sigma_\text{mul}(x))$ and $\xi_\text{add}(x)\sim\mathcal{N}(0,\sigma_\text{add}(x))$.

A total of $N$ points are sampled. Each sample at $x_i$, with $i=0,...,N-1$, is denoted by $g_j(x_i)$, with $j=0,...,n_i-1$, where $n_i$ is the total number of samples at $x_i$. 
We denote the estimate of the function $f$ at $x_i$ as
<br>
<br>
<center>$y_i = \frac{1}{n_i}\sum_{j=0}^{n_i-1} g_j(x_i)$.</center>
<br>
Moreover, the total number of samples is given by $n=\sum_i n_i$.

Let us define the uncertainty in the estimate of the mean at $x_i$ as
<br>
<br>
<center>$\Delta g_i = t_{\alpha,n_i-1} \sqrt{\frac{s_i^2}{n_i}}$,</center>
<br>
where $t_{\alpha,n_i-1}$ is the t-Student value and $s_i^2 = \frac{1}{n_i-1}\sum_{j=0}^{n_i-1}(g_j(x_i)-y_i)^2$ is the sample variance at $x_i$.
By using this definition, we ensure that $f(x_i)\in (y_i-\Delta g_i,y_i+\Delta g_i)$ with probability $1-2\alpha$.

---

### The strategy

Let us consider strategy 2, which ensures that $\Delta g_i < \delta$. 

However, when two points $x_i$ and $x_{i+1}$ are very close, e.g. when $x_{i+1}-x_i<\Delta g_i,\Delta g_{i+1}$, it does not make much sense to sample a new point inside the interval, since all the precision that is gained in the $x$-axis is wasted in the large uncertainty in the $y$-axis. Instead, it would make more sense to re-sample those points in order to decrease their uncertainty below the size of the interval. We want this to be applied locally (i.e. points away from the interval do not need to be re-sampled), so we cannot simply decrease $\delta$: we change the re-sampling condition to
<br>
<br>
<center>$\Delta g_i\cdot r_i > \delta$,</center>
<br>
where $r_i\geq1$ is the re-scaling factor at $x_i$. Initially, $r_i=1$, and it increases every time that $x_{i+1}-x_i<\Delta g_i,\Delta g_{i+1}$.

---

### Implementation comments

The learner stores a sorted dictionary with all the interval sizes. After a new sample is obtained (either new point or re-sampled), it checks whether
<br>
<br>
<center>$\min_{(x_i,x_{i+1})}(x_{i+1}-x_i) < \Delta g(x_i)\cdot R,\Delta g(x_{i+1})\cdot R$,</center>
<br>
where $R>1$. If this is true for $x_i$ and/or $x_{i+1}$, $r_i$ and/or $r_{i+1}$ will be increased, respectively.

---

### Pipeline summary of "learner.ask"

-If less than 2 points have been sampled:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\hookrightarrow$ Sample new point.

-Elif x is under-sampled:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\hookrightarrow$ Re-sample x.

-Elif &nbsp; $max_i(\Delta g_i \cdot r_i)>\delta$ &nbsp; ($r_i$ decreases with smaller interval sizes):
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\hookrightarrow$ Re-sample $x_i$.

-Else:
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\hookrightarrow$ Sample new point

&nbsp;

&nbsp;

&nbsp;